In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Epilepsy"
cohort = "GSE74571"

# Input paths
in_trait_dir = "../../input/GEO/Epilepsy"
in_cohort_dir = "../../input/GEO/Epilepsy/GSE74571"

# Output paths
out_data_file = "../../output/preprocess/Epilepsy/GSE74571.csv"
out_gene_data_file = "../../output/preprocess/Epilepsy/gene_data/GSE74571.csv"
out_clinical_data_file = "../../output/preprocess/Epilepsy/clinical_data/GSE74571.csv"
json_path = "../../output/preprocess/Epilepsy/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Serum Conditions Do Not Abolish The Tumorigenicity of Glioma Stem Cells"
!Series_summary	"The subset of GBM patient samples gives rise to adherent cultures even in sphere culture conditions. Most samples in this subset are tumorigenic and exhibit a hybrid expression profile when tested with the marker panel. Cultures from these samples have a predominantly mesenchymal character based on substrate adherence, morphology, differentiation potential and gene expression."
!Series_overall_design	"Total RNA isolated from glioblastoma stem cells (GSC) cultured as spheres was compared to that from adherent GSCs cultured in sphere culture conditions that exhibited both GSC and mesenchymal properties."
Sample Characteristics Dictionary:
{0: ['cell/tissue type: GBM cell culture in 10% serum; mixture of cells from adherent and sphere cultured', 'cell/tissue type: GBM cell culture in 1% serum; mixture of cells from adherent and sphere cultured', 'cell/tissue typ

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this appears to be gene expression data from GBM cells
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the sample characteristics dictionary
# For trait (Epilepsy): This dataset is about GBM (glioblastoma), not epilepsy
trait_row = None  # No Epilepsy data available

# For age: No age information in the sample characteristics
age_row = None

# For gender: No gender information in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    # Not needed since trait_row is None, but defining for completeness
    if pd.isna(value):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # For epilepsy, we would return 1 for epilepsy patients and 0 for controls
    # But this dataset doesn't have epilepsy data
    return None

def convert_age(value):
    # Not needed since age_row is None, but defining for completeness
    if pd.isna(value):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    # Not needed since gender_row is None, but defining for completeness
    if pd.isna(value):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save initial cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


SOFT file: ../../input/GEO/Epilepsy/GSE74571/GSE74571_family.soft.gz
Matrix file: ../../input/GEO/Epilepsy/GSE74571/GSE74571_series_matrix.txt.gz
Found the matrix table marker in the file.


Gene data shape: (47322, 36)
First 20 gene/probe identifiers:
['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209', 'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229', 'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253', 'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262']


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers start with "ILMN_" which are Illumina probe IDs, not human gene symbols
# These are microarray probe identifiers from Illumina platform that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Get a more complete view to understand the annotation structure
print("\nComplete sample of a few rows:")
print(gene_annotation.iloc[:3, :4].to_string())  # Show only first few columns for readability

# Examine the Symbol column which contains gene information
print("\nChecking for gene information in Symbol column:")
if 'Symbol' in gene_annotation.columns:
    sample_symbols = gene_annotation['Symbol'].head(5).tolist()
    print(f"Sample Symbol values: {sample_symbols}")
    
    # Use the library function to create the mapping
    mapping_data = get_gene_mapping(gene_annotation, 'ID', 'Symbol')
    
    # Print sample of the mapping to confirm
    print("\nSample of probe ID to gene symbol mappings:")
    print(mapping_data.head(10))
    
    # Check the size of the mapping data
    print(f"\nTotal number of probe-to-gene mappings: {len(mapping_data)}")
    
    # Check how many unique gene symbols we have
    unique_genes = mapping_data['Gene'].nunique()
    print(f"Number of unique gene symbols: {unique_genes}")
else:
    print("\nError: Could not find 'Symbol' column in the annotation data")



Gene annotation preview:
Columns in gene annotation: ['ID', 'Species', 'Source', 'Search_Key', 'Transcript', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Unigene_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Probe_Id', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], '

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Soft file and matrix file have already been identified in previous steps
# Extract gene annotation data from SOFT file for mapping
gene_annotation = get_gene_annotation(soft_file)

# 2. Get the gene mapping dataframe using the appropriate columns from gene annotation
# ID column contains probe identifiers matching those in gene expression data
# Symbol column contains the gene symbols
mapping_data = get_gene_mapping(gene_annotation, 'ID', 'Symbol')
print(f"Created mapping from {len(mapping_data)} probes to gene symbols")

# 3. Extract gene expression data from matrix file
gene_expression_data = get_genetic_data(matrix_file)
print(f"Extracted gene expression data with shape: {gene_expression_data.shape}")

# Apply the gene mapping to convert probe-level to gene-level expression
gene_data = apply_gene_mapping(gene_expression_data, mapping_data)
print(f"Converted to gene expression data with shape: {gene_data.shape}")

# Preview the resulting gene expression data
print("\nPreview of gene expression data:")
print(gene_data.head(3))


Created mapping from 44837 probes to gene symbols


Extracted gene expression data with shape: (47322, 36)
Converted to gene expression data with shape: (21463, 36)

Preview of gene expression data:
       GSM1923163  GSM1923164  GSM1923165  GSM1923166  GSM1923167  GSM1923168  \
Gene                                                                            
A1BG   156.612642  158.683792  141.065065  166.539842  147.699142  148.292042   
A1CF   199.563955  202.111744  204.275435  189.724489  197.069696  205.367373   
A26C3  200.364543  187.318372  195.010751  199.535568  196.223365  207.529118   

       GSM1923169  GSM1923170  GSM1923171  GSM1923172  ...  GSM1923189  \
Gene                                                   ...               
A1BG   140.293936  148.664011  142.755005  171.070572  ...  142.289219   
A1CF   199.349222  195.814469  186.430527  199.087704  ...  205.696066   
A26C3  201.087326  198.236513  195.836060  198.248079  ...  192.199217   

       GSM1923190  GSM1923191  GSM1923192  GSM1923193  GSM1923194  GSM192319

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data - respecting the analysis from Step 2
# From Step 2, we determined:
# trait_row = None  # No Epilepsy data available
# age_row = None
# gender_row = None
is_trait_available = trait_row is not None

# Skip clinical feature extraction when trait_row is None
if is_trait_available:
    try:
        # Load the clinical data from file
        soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        background_info, clinical_data = get_background_and_clinical_data(matrix_file)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            gender_row=gender_row,
            convert_gender=convert_gender,
            age_row=age_row,
            convert_age=convert_age
        )
        
        print(f"Extracted clinical data shape: {clinical_features.shape}")
        print("Preview of clinical data (first 5 samples):")
        print(clinical_features.iloc[:, :5])
        
        # Save the properly extracted clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical data: {e}")
        is_trait_available = False
else:
    print("No trait data (Epilepsy) available in this dataset based on previous analysis.")

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from GBM cell cultures, but no epilepsy phenotype information."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # Create an empty DataFrame for metadata purposes
    empty_df = pd.DataFrame()
    
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=empty_df,  # Empty dataframe for metadata
        note="Missing essential data components for linking: dataset contains gene expression data from GBM cell cultures, but no epilepsy phenotype information."
    )

Gene data shape before normalization: (21463, 36)
Gene data shape after normalization: (20259, 36)


Normalized gene data saved to ../../output/preprocess/Epilepsy/gene_data/GSE74571.csv
No trait data (Epilepsy) available in this dataset based on previous analysis.
Cannot proceed with data linking due to missing trait or gene data.
Abnormality detected in the cohort: GSE74571. Preprocessing failed.
